In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.endswith(".csv"):
            print(os.path.join(dirname, filename))

/kaggle/input/3-misogyny-meme-detection/3 Misogyny Meme Detection/Tamil/dev-20250101T183837Z-001/dev/dev.csv
/kaggle/input/3-misogyny-meme-detection/3 Misogyny Meme Detection/Tamil/train-20250101T183816Z-001/train/train.csv
/kaggle/input/3-misogyny-meme-detection/3 Misogyny Meme Detection/Tamil/test-20250101T183840Z-001/test/test.csv
/kaggle/input/3-misogyny-meme-detection/3 Misogyny Meme Detection/Malayalam/train-20250101T182940Z-001/train/train.csv
/kaggle/input/3-misogyny-meme-detection/3 Misogyny Meme Detection/Malayalam/dev-20250101T182941Z-001/dev/dev.csv
/kaggle/input/3-misogyny-meme-detection/3 Misogyny Meme Detection/Malayalam/test-20250101T182944Z-001/test/test.csv


# Loading Datasets

In [24]:
train_df=pd.read_csv("/kaggle/input/3-misogyny-meme-detection/3 Misogyny Meme Detection/Tamil/train-20250101T183816Z-001/train/train.csv")
train_df.head()

,image_id,labels,transcriptions
0,888,0,\nഈ ചാടി ഓടി നടക്കണ മനുഷ്യനാണോടാ നിങ്ങളിത്രേം ...
1,554,1,മലയാള സിനിമയുടെ ഭാവി വടറാണി ഇവൾ തന്നെ നല്ല കുഴ...
2,556,1,ഒന്ന് പെറ്റത് ആണെങ്കിലും .. മുലയും വയറും ചാടിയ...
3,484,1,ഓൺലൈൻ പരിചയപ്പെട്ടവനെ കളി തരാമെന്ന് പറഞ്ഞു അപ്...
4,370,0,കാമുകിയും അൺലിമിറ്റഡ് നെറ്റ് ഓഫറുംഉള്ള പയ്യന്റ...


In [4]:
val_df=pd.read_csv("/kaggle/input/3-misogyny-meme-detection/3 Misogyny Meme Detection/Tamil/dev-20250101T183837Z-001/dev/dev.csv")
val_df.head()

,image_id,labels,transcriptions
0,685,1,ചക്കക്കൊക്കെ ഇപ്പൊ എന്താ വില അല്ലെ എന്റെ പാർവ്...
1,796,0,ഗോവിന്ദൻ ഇഷ്\nED കള്ള കേസ് എടുത്തു സഹകരണമേഖലയെ...
2,641,1,ആരൊക്കെ എന്തൊക്കെ പറഞ്ഞാലും പെണ്ണ് ദിലീപ് ഷോയ്...
3,701,1,നമ്മുടെ എല്ലാരുടേം സ്കൂൾ ജീവിതത്തിൽ ഇങ്ങനൊരു ട...
4,926,0,വെള്ളം കുടിയ്ക്കാൻ പോയ ദില്ലിയ്ക്ക്\nചിക്കൻ ബി...


In [5]:
test_df=pd.read_csv("/kaggle/input/3-misogyny-meme-detection/3 Misogyny Meme Detection/Tamil/test-20250101T183840Z-001/test/test.csv")
test_df.head()

,image_id,transcriptions
0,954,ഇതിന് മാത്രം പിള്ളാരെ ഇവർക്കെങ്ങനെ cilşma??....
1,239,ഇത് ഞാൻ ചെറുതായിരുന്നപ്പോൾ ഇത് 5ആം ക്ലാസ്സ് വര...
2,61,എന്തൊക്കെ ആകിയിട്ടും ഒരു മെന വരുന്നില്ലലോ നിഖി...
3,984,ഇഞ്ചുറിക്കു ശേഷം ഇന്റർനാഷണൽ ഫുട്ബോൾലേക്കു തിരി...
4,774,* ഡോക്ടർ\nകണ്ണൻ സഹാവ്\nയ്യോ.. ഒരു equipment ഉള...


# Processing

In [6]:
import os
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from transformers import AutoModel, AutoTokenizer, AutoProcessor
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tqdm import tqdm

In [7]:
# Paths
TRAIN_IMAGE_FOLDER = "/kaggle/input/3-misogyny-meme-detection/3 Misogyny Meme Detection/Tamil/train-20250101T183816Z-001/train"
VAL_IMAGE_FOLDER = "/kaggle/input/3-misogyny-meme-detection/3 Misogyny Meme Detection/Tamil/dev-20250101T183837Z-001/dev"
TEST_IMAGE_FOLDER = "/kaggle/input/3-misogyny-meme-detection/3 Misogyny Meme Detection/Tamil/test-20250101T183840Z-001/test"

In [20]:
# Parameters
IMAGE_MODEL_NAME = "openai/clip-vit-base-patch32"  # Example vision model
TEXT_MODEL_NAME = "Hate-speech-CNERG/tamil-codemixed-abusive-MuRIL"  # Example Tamil BERT model
BATCH_SIZE = 8
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
# Ensure embedding directory exists
os.makedirs(EMBEDDINGS_DIR, exist_ok=True)

In [15]:
# Feature extraction classes for images and text
class ImageDataset(Dataset):
    def __init__(self, image_folder, df, processor):
        self.image_folder = image_folder
        self.df = df
        self.processor = processor
        self.transforms = Compose([
            Resize((224, 224)),
            ToTensor(),
            Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_id = self.df.iloc[idx]["image_id"]
        image_path = os.path.join(self.image_folder, f"{image_id}.jpg")
        image = self.transforms(self.processor(image_path))
        return image, image_id

In [16]:
class TextDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        transcription = self.df.iloc[idx]["transcriptions"]
        transcription = transcription if isinstance(transcription, str) else ""
        inputs = self.tokenizer(
            transcription, padding="max_length", truncation=True, max_length=128, return_tensors="pt"
        )
        return inputs, self.df.iloc[idx]["image_id"]

In [21]:
# Load Hugging Face models
vision_model = AutoModel.from_pretrained(IMAGE_MODEL_NAME).to(device)
vision_processor = AutoProcessor.from_pretrained(IMAGE_MODEL_NAME)
text_model = AutoModel.from_pretrained(TEXT_MODEL_NAME).to(device)
text_tokenizer = AutoTokenizer.from_pretrained(TEXT_MODEL_NAME)

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/950M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/523 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [28]:
from PIL import Image

def extract_image_embeddings(image_folder, df, save_path, model, processor):
    if os.path.exists(save_path):
        print(f"Embeddings already exist at {save_path}")
        return torch.load(save_path)

    embeddings = {}
    model.eval()
    with torch.no_grad():
        for idx, row in tqdm(df.iterrows(), desc="Extracting image embeddings", total=len(df)):
            image_id = row["image_id"]
            image_path = os.path.join(image_folder, f"{image_id}.jpg")
            
            if not os.path.exists(image_path):
                print(f"Image not found: {image_path}")
                continue

            try:
                # Open image with PIL
                image = Image.open(image_path).convert("RGB")
                # Preprocess image
                inputs = processor(images=image, return_tensors="pt").to(device)
                # Extract embeddings
                outputs = model.get_image_features(**inputs)
                embeddings[image_id] = outputs.cpu()
            except Exception as e:
                print(f"Error processing image {image_id}: {e}")
                continue

    torch.save(embeddings, save_path)
    return embeddings

In [29]:
# Extract embeddings
train_image_embeddings = extract_image_embeddings(
    TRAIN_IMAGE_FOLDER, train_df, "train_image_embeddings.pt", vision_model, vision_processor
)
val_image_embeddings = extract_image_embeddings(
    VAL_IMAGE_FOLDER, val_df, "val_image_embeddings.pt", vision_model, vision_processor
)
test_image_embeddings = extract_image_embeddings(
    TEST_IMAGE_FOLDER, test_df, "test_image_embeddings.pt", vision_model, vision_processor
)

Extracting image embeddings: 100%|██████████| 200/200 [00:08<00:00, 23.16it/s]


In [30]:
def extract_text_embeddings(df, save_path, model, tokenizer):
    if os.path.exists(save_path):
        print(f"Embeddings already exist at {save_path}")
        return torch.load(save_path)

    embeddings = {}
    model.eval()
    with torch.no_grad():
        for idx, row in tqdm(df.iterrows(), desc="Extracting text embeddings", total=len(df)):
            transcription = row["transcriptions"]
            transcription = transcription if isinstance(transcription, str) else ""
            
            # Tokenize the text
            inputs = tokenizer(
                transcription, padding="max_length", truncation=True, max_length=128, return_tensors="pt"
            )
            inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to GPU/CPU
            
            # Extract embeddings
            outputs = model(**inputs)
            embeddings[row["image_id"]] = outputs.pooler_output.cpu()

    torch.save(embeddings, save_path)
    return embeddings

In [31]:
train_text_embeddings = extract_text_embeddings(
    train_df, "train_text_embeddings.pt", text_model, text_tokenizer
)
val_text_embeddings = extract_text_embeddings(
    val_df, "val_text_embeddings.pt", text_model, text_tokenizer
)
test_text_embeddings = extract_text_embeddings(
    test_df, "test_text_embeddings.pt", text_model, text_tokenizer
)

Extracting text embeddings: 100%|██████████| 200/200 [00:01<00:00, 101.33it/s]
